In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mk=2

# Initial 

In [ ]:
import scipy.io
import numpy as np
file_name = '/content/drive/MyDrive/2022_SUMMER/Research_Burger/CODE/TranslatePythonCode/'
#file_name = '/content/drive/MyDrive/tamu/BurgerResults/'
mat = scipy.io.loadmat(file_name + 'canadafastfood_resorted.mat')
data = mat['data']#[np.where(mtype_int==1)]
TimePeriod = len(np.unique(data[:,1]))
SamplePath = int(len(data)/TimePeriod)
print(TimePeriod,SamplePath)
print(data[:,0].reshape([TimePeriod,SamplePath]).T)

#% Labeling
MT = len(data)          #% Size of the dataset (M x T = 400 x 35 = 14000).#clusterid = np.int_(data[:,0])      % Unique market ID.
year = np.int_(data[:,1]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_aw  = np.int_(data[:,2]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_bk  = np.int_(data[:,3]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_hvy = np.int_(data[:,4]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_mcd = np.int_(data[:,5]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_wdy = np.int_(data[:,6]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_aw  = np.int_(data[:,7]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_bk  = np.int_(data[:,8]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_hvy = np.int_(data[:,9]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_mcd = np.int_(data[:,10]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_wdy = np.int_(data[:,11]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_aw  = np.int_(data[:,12]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_bk  = np.int_(data[:,13]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_hvy = np.int_(data[:,14]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_mcd = np.int_(data[:,15]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_wdy = np.int_(data[:,16]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_aw  = np.int_(data[:,17]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_bk  = np.int_(data[:,18]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_hvy = np.int_(data[:,19]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_mcd = np.int_(data[:,20]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_wdy = np.int_(data[:,21]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
pop = data[:,23].reshape([TimePeriod,SamplePath]).T.reshape(-1)
val = data[:,24].reshape([TimePeriod,SamplePath]).T.reshape(-1)
inc = data[:,25].reshape([TimePeriod,SamplePath]).T.reshape(-1)
fpop = data[:,27].reshape([TimePeriod,SamplePath]).T.reshape(-1)
finc = data[:,28].reshape([TimePeriod,SamplePath]).T.reshape(-1)
fval = data[:,29].reshape([TimePeriod,SamplePath]).T.reshape(-1)
mktfe = data[:,30].reshape([TimePeriod,SamplePath]).T.reshape(-1) #% Market fixed effect estimates (from 130913_alaToivanenWaterson3_Fixedeffect3quantile.csv/dta)
tertile = np.int_(data[:,31]).reshape([TimePeriod,SamplePath]).T.reshape(-1)       #% Market type initial guess (from 130913_alaToivanenWaterson3_Fixedeffect3quantile.csv/dta)

35 400
[[  1.   1.   1. ...   1.   1.   1.]
 [  2.   2.   2. ...   2.   2.   2.]
 [  3.   3.   3. ...   3.   3.   3.]
 ...
 [398. 398. 398. ... 398. 398. 398.]
 [399. 399. 399. ... 399. 399. 399.]
 [400. 400. 400. ... 400. 400. 400.]]


In [ ]:
def quantile(x,p):
  x = np.sort(x)
  n = len(x)
  x = np.hstack([x[0], x,x[-1]])
  i = p*n+1.5
  iu = int(np.ceil(i))
  il = int(np.floor(i))
  d = i-il
  qq = x[il-1]*(1-d)+x[iu-1]*d
  return qq

#% Number of own shops (in state space; capped at 3), from the perspective of each firm
Ni_aw = (N_aw <= 3)*N_aw + (N_aw > 3)*3
Ni_bk = (N_bk <= 3)*N_bk + (N_bk > 3)*3
Ni_hvy = (N_hvy <= 3)*N_hvy + (N_hvy > 3)*3
Ni_mcd = (N_mcd <= 3)*N_mcd + (N_mcd > 3)*3
Ni_wdy = (N_wdy <= 3)*N_wdy + (N_wdy > 3)*3

#% Number of rival shops (in state space; capped at 3), from the perspective of each firm
Nj_aw = N_bk + N_hvy + N_mcd + N_wdy
Nj_bk = N_aw + N_hvy + N_mcd + N_wdy
Nj_hvy = N_aw + N_bk + N_mcd + N_wdy
Nj_mcd = N_aw + N_bk + N_hvy + N_wdy
Nj_wdy = N_aw + N_bk + N_hvy + N_mcd
Nj_aw = (Nj_aw <= 3)*Nj_aw + (Nj_aw > 3)*3     
Nj_bk = (Nj_bk <= 3)*Nj_bk + (Nj_bk > 3)*3
Nj_hvy = (Nj_hvy <= 3)*Nj_hvy + (Nj_hvy > 3)*3
Nj_mcd = (Nj_mcd <= 3)*Nj_mcd + (Nj_mcd > 3)*3
Nj_wdy = (Nj_wdy <= 3)*Nj_wdy + (Nj_wdy > 3)*3

#% Number of pop,val,inc (in state space; capped at 3)
pop25 = quantile(pop,0.25)
val25 = quantile(val,0.25)   
inc25 = quantile(inc,0.25)  
pop50 = quantile(pop,0.5) 
val50 = quantile(val,0.5)   
inc50 = quantile(inc,0.5)  
pop75 = quantile(pop,0.75) 
val75 = quantile(val,0.75)   
inc75 = quantile(inc,0.75)   
disc_pop = np.int_(pop > pop25) + np.int_(pop > pop50) + np.int_(pop > pop75)  
disc_val = np.int_(val > val25) + np.int_(val > val50) + np.int_(val > val75)  
disc_inc = np.int_(inc > inc25) + np.int_( inc > inc50) + np.int_(inc > inc75)

#% Define state variables for each firm
RHS_aw = np.vstack([Ni_aw, Nj_aw, disc_pop,disc_val, disc_inc]).T#;         % State for A & W.      
RHS_bk = np.vstack([Ni_bk, Nj_bk, disc_pop,disc_val, disc_inc]).T#;         % State for Burger King.      
RHS_hvy = np.vstack([Ni_hvy, Nj_hvy, disc_pop,disc_val, disc_inc]).T#;      % State for Harvey's.      
RHS_mcd = np.vstack([Ni_mcd, Nj_mcd, disc_pop, disc_val,disc_inc]).T#;      % State for McDonald's.      
RHS_wdy = np.vstack( [Ni_wdy, Nj_wdy, disc_pop,disc_val, disc_inc]).T#;

In [ ]:
stateID_aw = Ni_aw*4**4+Nj_aw*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_bk = Ni_bk*4**4+Nj_bk*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_hvy = Ni_hvy*4**4+Nj_hvy*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_mcd = Ni_mcd*4**4+Nj_mcd*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_wdy = Ni_wdy*4**4+Nj_wdy*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_other = np.hstack([stateID_aw,stateID_bk,stateID_hvy,stateID_wdy])

ai_aw = -1*np.int_(a_aw<0) + 0*np.int_(a_aw==0) + 1*np.int_(a_aw>0)+1
ai_bk = -1*np.int_(a_bk<0) + 0*np.int_(a_bk==0) + 1*np.int_(a_bk>0) +1
ai_hvy = -1*np.int_(a_hvy<0) + 0*np.int_(a_hvy==0) + 1*np.int_(a_hvy>0)+1
ai_mcd = -1*np.int_(a_mcd<0) + 0*np.int_(a_mcd==0) + 1*np.int_(a_mcd>0)+1
ai_wdy = -1*np.int_(a_wdy<0) + 0*np.int_(a_wdy==0) + 1*np.int_(a_wdy>0)+1
ai_other = np.hstack([ai_aw,ai_bk,ai_hvy,ai_wdy])

In [ ]:
numstates = 5 # ni,nj,pop,val,inc
states_full = np.arange(4**numstates)
Ni = states_full//(4**(numstates-1))                       #Number of own outlets: {0,1,2,3+}.
Nj = states_full%(4**(numstates-1))//(4**(numstates-2))    #Number of rival outlets: {0,1,2,3+}.
dz1 = states_full%(4**(numstates-2))//(4**(numstates-3))   #Discretized population state: {0,1,2,3}.
dz2 = states_full%(4**(numstates-3))//(4**(numstates-4))   #Discretized property value state: {0,1,2,3}.
dz3 = states_full%(4**(numstates-4))                       #Discretized income state: {0,1,2,3}.
print(np.sum(np.abs(Ni*4**4+Nj*4**3+dz1*4**2+dz2*4**1+dz3-states_full)))
states = np.stack([Ni,Nj,dz1,dz2,dz3],axis=1)

0


In [ ]:
import pandas as pd
filename = '/content/drive/MyDrive/2022_SUMMER/Research_Burger/Picture/'#file_name#
data_extra = pd.read_csv(filename+'ExtraData.csv')
gdp = data_extra['GDP_Grwoth_Rate']
gdp50 = quantile(gdp,0.5)  
obser = 2
disc_gdp = np.int_(gdp > 0) #+ np.int_(gdp > gdp75) #np.int_(gdp>gdp50)#
trans_gdp = np.zeros([obser,obser],dtype=int)

for i in range(34):
  trans_gdp[disc_gdp[i],disc_gdp[i+1]] +=1
print(trans_gdp)
trans_gdp = trans_gdp/np.kron(np.ones([1,obser]),np.sum(trans_gdp,axis=1).reshape([-1,1]))
print(trans_gdp)
disc_gdp_fw = ((np.repeat(disc_gdp[1:TimePeriod+1],SamplePath).reshape([TimePeriod,SamplePath]).T).reshape(-1))[np.where(tertile==mk)]
disc_gdp_bk = ((np.repeat(disc_gdp[0:TimePeriod],SamplePath).reshape([TimePeriod,SamplePath]).T).reshape(-1))[np.where(tertile==mk)]
disc_gdp = (np.repeat(disc_gdp[0:TimePeriod],SamplePath).reshape([TimePeriod,SamplePath]).T).reshape(-1)
l12 = 0#np.sum(np.log(trans_gdp[disc_gdp_bk,disc_gdp_fw]))
print(l12)

[[ 1  2]
 [ 2 29]]
[[0.33333333 0.66666667]
 [0.06451613 0.93548387]]
0


# Function

##1. Global transtion F_Ni, F_dz1,F_dz2,F_dz3

In [ ]:
I = 2 #MCD and Other homogenous
J = 3 #Action: 0,1,2
NS = 4**(numstates)
NX = 1

In [ ]:
#% F_Ni: Own # of shops changes deterministically, as follows
F_Ni = np.zeros([I,J,NX,NS,NS])#;
#for i in range(NX): 
F_Ni[0,0] = np.kron(np.array([[1]*4**4+[0]*4**4 + [0] *4**4+[0]*4**4]*4**4 +\
                          [[1]*4**4+[0]*4**4 + [0] *4**4+[0]*4**4]*4**4 + \
                          [[0]*4**4+[1]*4**4 + [0] *4**4+[0]*4**4]*4**4 + \
                          [[0]*4**4+[0]*4**4 + [1] *4**4+[0]*4**4]*4**4 ),np.ones([NX,1,1]))
F_Ni[0,1] =np.kron(np.array([[1]*4**4+[0]*4**4 + [0] *4**4+[0]*4**4]*4**4 +\
                          [[0]*4**4+[1]*4**4 + [0] *4**4+[0]*4**4]*4**4 + \
                          [[0]*4**4+[0]*4**4 + [1] *4**4+[0]*4**4]*4**4 + \
                          [[0]*4**4+[0]*4**4 + [0] *4**4+[1]*4**4]*4**4 ),np.ones([NX,1,1]))
F_Ni[0,2] = np.kron(np.array([[0]*4**4+[1]*4**4 + [0] *4**4+[0]*4**4]*4**4 +\
                          [[0]*4**4+[0]*4**4 + [1] *4**4+[0]*4**4]*4**4 + \
                          [[0]*4**4+[0]*4**4 + [0] *4**4+[1]*4**4]*4**4 + \
                          [[0]*4**4+[0]*4**4 + [0] *4**4+[1]*4**4]*4**4 ),np.ones([NX,1,1]))
F_Ni[1] = F_Ni[0]
# F_Ni[1:5,0,:,:] = np.stack([F_Ni[0,0,:,:],F_Ni[0,0,:,:],F_Ni[0,0,:,:],F_Ni[0,0,:,:]],axis=0)
# F_Ni[1:5,1,:,:] = np.stack([F_Ni[0,1,:,:],F_Ni[0,1,:,:],F_Ni[0,1,:,:],F_Ni[0,1,:,:]],axis=0)
# F_Ni[1:5,2,:,:] = np.stack([F_Ni[0,2,:,:],F_Ni[0,2,:,:],F_Ni[0,2,:,:],F_Ni[0,2,:,:]],axis=0)

disc_pop_mx = disc_pop[tertile==mk].reshape([-1,TimePeriod])
disc_pop_bk = disc_pop_mx[:,0:TimePeriod-1].reshape(-1)
disc_pop_fw = disc_pop_mx[:,1:TimePeriod].reshape(-1)
disc_val_mx = disc_val[tertile==mk].reshape([-1,TimePeriod])
disc_val_bk = disc_val_mx[:,0:TimePeriod-1].reshape(-1)
disc_val_fw = disc_val_mx[:,1:TimePeriod].reshape(-1)
disc_inc_mx = disc_inc[tertile==mk].reshape([-1,TimePeriod])
disc_inc_bk = disc_inc_mx[:,0:TimePeriod-1].reshape(-1)
disc_inc_fw = disc_inc_mx[:,1:TimePeriod].reshape(-1)

#% F_dz: Make simple & intuitive (4x4) versions for exposition purposes
fz1_4x4 = np.zeros([4,4])#;
fz2_4x4 = np.zeros([4,4])#;
fz3_4x4 = np.zeros([4,4])#;

for x0 in range(4):
    for x1 in range(4):
        numer1 = np.sum((disc_pop_fw == x1) * (disc_pop_bk == x0))#;
        numer2 = np.sum((disc_val_fw == x1) * (disc_val_bk == x0))#;
        numer3 = np.sum((disc_inc_fw == x1) * (disc_inc_bk == x0))#;
        denom1 = np.sum((disc_pop_bk == x0))#;
        denom2 = np.sum((disc_val_bk == x0))#;
        denom3 = np.sum((disc_inc_bk == x0))#;

        fz1_4x4[x0,x1] = numer1 / denom1#;
        fz2_4x4[x0,x1] = numer2 / denom2#;
        fz3_4x4[x0,x1] = numer3 / denom3#;

for i in range(4):
  print("&{}&{:.4f}&{:.4f}&{:.4f}&{:.4f}\\\\".format(i,fz1_4x4[i][0],fz1_4x4[i][1],fz1_4x4[i][2],fz1_4x4[i][3]))
print('------------------------------')
for i in range(4):
  print("&{}&{:.4f}&{:.4f}&{:.4f}&{:.4f}\\\\".format(i,fz2_4x4[i][0],fz2_4x4[i][1],fz2_4x4[i][2],fz2_4x4[i][3]))
print('------------------------------')
for i in range(4):
  print("&{}&{:.4f}&{:.4f}&{:.4f}&{:.4f}\\\\".format(i,fz3_4x4[i][0],fz3_4x4[i][1],fz3_4x4[i][2],fz3_4x4[i][3]))
print('------------------------------')
l11 = np.sum(np.log(fz1_4x4[disc_pop_bk,disc_pop_fw])+np.log(fz2_4x4[disc_val_bk,disc_val_fw])+np.log(fz3_4x4[disc_inc_bk,disc_inc_fw]))
print(l11,l11-1447.116432028803,l11+l12)
#print(fz1_4x4)
#print(fz2_4x4)
#print(fz3_4x4)

F_dz1 = np.kron((fz1_4x4[dz1,:])[:,dz1],np.ones([I,J,NX,1,1]))#; % The transition pattern is common across (i,j,mtype)
F_dz2 = np.kron((fz2_4x4[dz2,:])[:,dz2],np.ones([I,J,NX,1,1]))#; % The transition pattern is common across (i,j,mtype)
F_dz3 = np.kron((fz3_4x4[dz3,:])[:,dz3],np.ones([I,J,NX,1,1]))#; % The transition pattern is common across (i,j,mtype)



&0&0.9794&0.0206&0.0000&0.0000\\
&1&0.0000&0.9529&0.0471&0.0000\\
&2&0.0000&0.0000&0.9582&0.0418\\
&3&0.0000&0.0000&0.0012&0.9988\\
------------------------------
&0&0.9346&0.0654&0.0000&0.0000\\
&1&0.0008&0.9169&0.0822&0.0000\\
&2&0.0000&0.0063&0.9003&0.0934\\
&3&0.0000&0.0000&0.0147&0.9853\\
------------------------------
&0&0.9124&0.0876&0.0000&0.0000\\
&1&0.0083&0.8632&0.1286&0.0000\\
&2&0.0000&0.0174&0.9040&0.0786\\
&3&0.0000&0.0000&0.0183&0.9817\\
------------------------------
-3017.3878228464882 -4464.504254875292 -3017.3878228464882


##2. Dynamix and SigmaLambda

In [ ]:
import itertools
dim_hstate = 2
#dim_observe = 2
def Dynamic(theta23):
  trans_val = theta23[0:2]
  #obser_val = theta23[2:]

  trans = np.zeros([dim_hstate,dim_hstate]) #s_t^h,s_{t+1}^h
  trans[:,0] = trans_val
  trans[:,1] = 1-trans[:,0]


  # obser = np.zeros([dim_hstate,dim_observe]) #s_{t+1}^h,z_{t+1}
  # obser[:,0] =[obser_val,1-obser_val]# obser_val#
  # obser[:,1] = 1-obser[:,0]
  return trans#,obser

#print(Dynamic(trans_gdp[:,0]))
theta23 = trans_gdp[:,0]
trans = Dynamic(theta23)
print(trans)
# #print(Dynamic(np.array([0.95, 0.05]+ [0.9, 0.01])))  
# def SigmaLambda(theta23,z_old = None,x_old=None,T=None,num_discrete = NX):
#   trans = Dynamic(theta23)
#   if T==None: #generate function for Q function
#     x_old =  np.linspace(0,1,num=num_discrete) #z_old
#     x_new = np.zeros([num_discrete,dim_hstate]) #z_old,z_new
#     sigma = np.zeros([num_discrete,dim_hstate]) #z_old,z_new

#     for z_prime in range(dim_hstate):
#       x_temp = (x_old*trans[0,0] + (1-x_old) * trans[1,0])*obser[0,z_prime]
#       sigma[:,z_prime] = x_temp + (x_old*trans[0,1] + (1-x_old) * trans[1,1])*obser[1,z_prime]
#       sigma_nonzero = np.where(sigma[:,z_prime]!=0)
#       x_new[:,z_prime][sigma_nonzero] = x_temp[sigma_nonzero]/sigma[:,z_prime][sigma_nonzero]

#   else: #generate whole blief in recover process
#     x_new = np.zeros([T,len(x_old)])
#     sigma = np.zeros([T-1,len(x_old)])
#     x_new[0] = x_old
#     for t in range(T-1):
#       z_prime = z_old[t+1,:]
#       x_temp = (x_new[t]*trans[0,0]+(1-x_new[t])*trans[1,0])*obser[0,z_prime]
#       sigma[t] = x_temp + (x_new[t]*trans[0,1]+(1-x_new[t])*trans[1,1])*obser[1,z_prime]
#       sigma_nonzero = np.where(sigma[t]!=0)
#       x_new[t+1][sigma_nonzero] = x_temp[sigma_nonzero]/sigma[t][sigma_nonzero]

#   belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
#   belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
#   iterpolate = np.zeros(belief_f.shape)
#   iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      

#   return sigma,x_new,[iterpolate,np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))]

# theta23 = np.array([0.94 ,0.00813008,0.9])#, 0.01] )
# sigma,_,(iterp,iterf,iterc) = SigmaLambda(theta23)
# # sigmaP = (sigma[:,:,dz2])[:,dz2]
# # sigmaP = np.stack([np.stack([sigmaP for j in range(J)],axis=0) for i in range(I)],axis=0)
# # iterpP = (iterp[:,:,dz2])[:,dz2]
# # iterfP = (iterf[:,:,dz2])[:,dz2]
# # itercP = (iterc[:,:,dz2])[:,dz2]

[[0.33333333 0.66666667]
 [0.06451613 0.93548387]]


## 3. Generate P(nj|policy)

In [ ]:
F_fix = F_Ni * F_dz1 * F_dz2 *F_dz3 #agent, action, z_old, s_old, s_new

In [ ]:
# % Calculate F from P (& global transition matrices F_Ni, F_dz1, F_dz2) 
import time
def updateF(P):
  #% For a given combination of (Ni, dz1, dz2, mtype), make the state indexes (x, x0, x1, x2, x3)
  #% Beliefs: mapping from today's state (Ni,Nj,dz1,dz2) to tomorrow's Nj = {0,1,2,3}
  fnj_mcd = np.zeros([NX,NS,4])#;      % McDonald's belief
  fnj_other = np.zeros([NX,NS,4])#;    % Other 4 chains beliefs

  nj = 0#;         % If today's # of rivals = 0
  x =  (4**4)*Ni + (4**3)*nj + (4**2)*dz1 + 4*dz2+dz3#;  % index of today's own state
  x0 = (4**4)*0 + (4**3)*Ni + (4**2)*dz1 + 4*dz2+dz3#;  % index of today's state for rivals with 0 shops

  fnj_mcd[:,x,0] = P[1,1][:,x0]**4#;                               % Prob(Nj'= 0) from McD's perspective
  fnj_mcd[:,x,1] = 4 * P[1,2][:,x0] * (P[1,1][:,x0]**3)#;       % Prob(Nj'= 1) from McD's perspective
  fnj_mcd[:,x,2] = 6 * (P[1,2][:,x0]**2) * (P[1,1][:,x0]**2)#;   % Prob(Nj'= 2) from McD's perspective
  fnj_mcd[:,x,3] = 4 * (P[1,2][:,x0]**3) * P[1,1][:,x0]#;       % Prob(Nj'= 3) from McD's perspective
  
  fnj_other[:,x,0] = P[0,1][:,x0] * (P[1,1][:,x0]**3)#;         % Prob(Nj'= 0) etc. from Others' perspectives
  fnj_other[:,x,1] = P[0,2][:,x0] * (P[1,1][:,x0]**3) + P[0,1][:,x0] * 3 * P[1,2][:,x0] * (P[1,1][:,x0]**2)#;  
  fnj_other[:,x,2] = P[0,2][:,x0] * 3 * P[1,2][:,x0] * (P[1,1][:,x0]**2) + P[0,1][:,x0] * 3 * (P[1,2][:,x0]**2) * P[1,1][:,x0]
  fnj_other[:,x,3] = P[0,2][:,x0] * 3 * (P[1,2][:,x0]**2) * P[1,1][:,x0] + P[0,1][:,x0] * (P[1,2][:,x0]**3)
  
  nj = 1#;         % If today's # of rivals = 1                               
  x =  (4**4)*Ni + (4**3)*nj + (4**2)*dz1 +4*dz2+dz3#;  % index of today's own state
  x0 = (4**4)*0 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 0 shops
  x1 =  (4**4)*1 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 1 shops
  
  fnj_mcd[:,x,0] = P[1,0][:,x1] * (P[1,1][:,x0]**3)#;
  fnj_mcd[:,x,1] = P[1,1][:,x1] * (P[1,1][:,x0]**3)#;
  fnj_mcd[:,x,2] = P[1,2][:,x1] * (P[1,1][:,x0]**3) + P[1,1][:,x1] * 3 * P[1,2][:,x0] * (P[1,1][:,x0]**2)#;
  fnj_mcd[:,x,3] = P[1,2][:,x1] * 3 * P[1,2][:,x0] * (P[1,1][:,x0]**2) + P[1,1][:,x1] * 3 * (P[1,2][:,x0]**2) * P[1,1][:,x0]
  
  fnj_other[:,x,0] = .5 * (P[0,0][:,x1] * P[1,1][:,x0]**3) + .5 * (P[0,1][:,x0] * P[1,0][:,x1] * (P[1,1][:,x0]**2))
  fnj_other[:,x,1] = .5 * (P[0,1][:,x1] * P[1,1][:,x0]**3) + .5 * (P[0,1][:,x0] * P[1,1][:,x1] * (P[1,1][:,x0]**2))
  fnj_other[:,x,2] = .5 * (P[0,2][:,x1] * (P[1,1][:,x0]**3) + P[0,1][:,x1] * 3 * P[1,2][:,x0] * (P[1,1][:,x0]**2))+\
                              .5 * (P[0,2][:,x0] * P[1,1][:,x1] * (P[1,1][:,x0]**2) +\
                        P[0,1][:,x0] * (P[1,2][:,x1] * (P[1,1][:,x0]**2) + P[1,1][:,x1] * 2 * P[1,2][:,x0] * P[1,1][:,x0]))
  fnj_other[:,x,3] = .5 * (P[0,2][:,x1] * 3 * P[1,2][:,x0] * (P[1,1][:,x0]**2)\
                        + P[0,1][:,x1] * 3 * (P[1,2][:,x0]**2) * P[1,1][:,x0])\
                        + .5 * (P[0,2][:,x0] * P[1,2][:,x1] * (P[1,1][:,x0]**2)\
                      + P[0,2][:,x0] * P[1,1][:,x1] * 2 * P[1,2][:,x0] * P[1,1][:,x0]\
                      + P[0,1][:,x0] * P[1,2][:,x1] * 2 * P[1,2][:,x0] * P[1,1][:,x0]\
                      + P[0,1][:,x0] * P[1,1][:,x1] * (P[1,2][:,x0]**2))

  nj = 2#;         % If today's # of rivals = 2
  x = (4**4)*Ni + (4**3)*nj + (4**2)*dz1 +4*dz2+dz3#;  % index of today's own state
  x0 = (4**4)*0 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 0 shops
  x1 = (4**4)*1 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 1 shops
  x2 = (4**4)*2 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 2 shops
  
  fnj_mcd[:,x,0] = (P[1,0][:,x1]**2) * (P[1,1][:,x0]**2)
  fnj_mcd[:,x,1] = (1/3) * (P[1,0][:,x2] * (P[1,1][:,x0]**3)) + (2/3) * (P[1,0][:,x1] * P[1,1][:,x1] * (P[1,1][:,x0]**2))
  fnj_mcd[:,x,2] = (1/3) * (P[1,1][:,x2] * (P[1,1][:,x0]**3)) + (2/3) * ((P[1,1][:,x1]**2) * (P[1,1][:,x0]**2))
  fnj_mcd[:,x,3] = (1/3) * (P[1,2][:,x2] * (P[1,1][:,x0]**3)  + (P[1,1][:,x2] * 3 * P[1,2][:,x0] * (P[1,1][:,x0]**2)))
  
  fnj_other[:,x,0] = .5 * 0 + .5 * (P[0,0][:,x1] * P[1,0][:,x1] * (P[1,1][:,x0]**2))
  fnj_other[:,x,1] = .5 * (P[0,0][:,x2] * (P[1,1][:,x0]**3))\
      + .5 * (P[0,0][:,x1] * P[1,1][:,x1] * (P[1,1][:,x0]**2)\
      + P[0,1][:,x1] * P[1,0][:,x1] * (P[1,1][:,x0]**2))
  fnj_other[:,x,2] = .5 * (P[0,1][:,x2] * (P[1,1][:,x0]**3))\
      + .5 * (P[0,1][:,x1] * P[1,1][:,x1] * (P[1,1][:,x0]**2))
  fnj_other[:,x,3] = .5 * (P[0,2][:,x2] * (P[1,1][:,x0]**3)\
      + P[0,1][:,x2] * 3 * P[1,2][:,x0] * P[1,1][:,x0])\
      + .5 * (P[0,2][:,x1] * P[1,1][:,x1] * (P[1,1][:,x0]**2)\
      + P[0,1][:,x1] * P[1,2][:,x1] * (P[1,1][:,x0]**2)\
      + P[0,1][:,x1] * P[1,1][:,x1] * 2 * P[1,2][:,x0] * P[1,1][:,x0])
  
  nj = 3#;         % If today's # of rivals = 3
  x = (4**4)*Ni + (4**3)*nj + (4**2)*dz1 +4*dz2+dz3#;  % index of today's own state
  x0 = (4**4)*0 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 0 shops
  x1 = (4**4)*1 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 1 shops
  x2 = (4**4)*2 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 2 shops
  x3 = (4**4)*3 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 3 shops
  
  fnj_mcd[:,x,0] = (2/3) * (P[1,0][:,x1]**3) * P[1,1][:,x0]
  fnj_mcd[:,x,1] = (2/3) * (P[1,0][:,x1]**2) * P[1,1][:,x1] + (1/3) * P[1,0][:,x2] * P[1,0][:,x1] * (P[1,1][:,x0]**2)
  fnj_mcd[:,x,2] = (2/3) * P[1,0][:,x1] * (P[1,1][:,x1]**2) + (1/3) * (P[1,0][:,x2] * P[1,1][:,x1] * (P[1,1][:,x0]**2)\
      + P[1,1][:,x2] * P[1,0][:,x1] * (P[1,1][:,x0]**2))
  fnj_mcd[:,x,3] = (2/3) * (P[1,1][:,x1]**3) * P[1,1][:,x0] + (1/3) * P[1,1][:,x2] * P[1,1][:,x1] * (P[1,1][:,x0]**2)
  
  fnj_other[:,x,0] = .25 * (P[0,0][:,x1] * (P[1,0][:,x1]**2) * P[1,1][:,x0])
  fnj_other[:,x,1] = .25 * 0 + .25 * (P[0,0][:,x2] * P[1,0][:,x1] * (P[1,1][:,x0]**2))\
      + .25 * (P[0,0][:,x1] * 2 * P[1,0][:,x1] * P[1,1][:,x1] * P[1,1][:,x0]\
      + P[0,1][:,x1] * (P[1,0][:,x1]**2) * P[1,1][:,x0])\
      + .25 * (P[0,0][:,x1] * P[1,0][:,x2] * (P[1,1][:,x0]**2))
  fnj_other[:,x,2] = .25 * (P[0,0][:,x3] * (P[1,1][:,x0]**3))\
      + .25 * (P[0,0][:,x2] * P[1,1][:,x1] * (P[1,1][:,x0]**2)\
      + P[0,1][:,x2] * P[1,0][:,x1] * (P[1,1][:,x0]**2))\
      + .25 * (P[0,0][:,x1] * (P[1,1][:,x1]**2) * P[1,1][:,x0]\
      + P[0,1][:,x1] * 2 * P[1,0][:,x1] * P[1,1][:,x1] * P[1,1][:,x0])\
      + .25 * (P[0,0][:,x1] * P[1,1][:,x2] * (P[1,1][:,x0]**2)\
      + P[0,1][:,x1] * P[1,0][:,x2] * (P[1,1][:,x0]**2))
  fnj_other[:,x,3] = .25 * (P[0,1][:,x3] * (P[1,1][:,x0]**3))\
      + .25 * (P[0,1][:,x2] * P[1,1][:,x1] * (P[1,1][:,x0]**2))\
      + .25 * (P[0,1][:,x1] * (P[1,1][:,x1]**2) * P[1,1][:,x0])\
      + .25 * (P[0,1][:,x1] * P[1,1][:,x2] * (P[1,1][:,x0]**2))

  F_Nj = np.ones([I,J,NX,NS,NS])
  Denom_mcd = np.matmul(fnj_mcd,np.ones([NX,4,4]))#np.kron(np.reshape(np.sum(fnj_mcd,axis=2),[NX,-1,1]),np.ones([1,1,4]))#;         % Sum of each row
  fnj_mcd = fnj_mcd / Denom_mcd#;                 % Make sure each row sums up to 1
  fnj_mcd[np.isnan(fnj_mcd)] = 0#;                    % Replace NaN with 0
  F_Nj[0,0] = fnj_mcd[:,:,Nj]

  Denom_other = np.matmul(fnj_other,np.ones([NX,4,4]))#np.kron(np.reshape(np.sum(fnj_other,axis=2),[NX,-1,1]),np.ones([1,1,4]))#;             % Sum of each row
  fnj_other = fnj_other / Denom_other#;                   % Make sure each row sums up to 1
  fnj_other[np.isnan(fnj_other)] = 0#;                        % Replace NaN with 0
  F_Nj[1,0] = fnj_other[:,:,Nj]
  #F_Nj[2:5,j,:,:] = np.stack([F_Nj[1,j,:,:] for copy in range(3)],axis=0)#;    % Other 4 chains are symmetric

  for j in range(1,J):    
      F_Nj[:,j] = F_Nj[:,0]    
  return F_Nj*F_fix

P_new = 1/J*np.ones([I,J,NX,NS])#softmax(np.arange(I*J*NS).reshape([I,J,NS]),axis=1)
F_new = updateF(P_new.copy())
#np.sum(F_new!=0)

##4.Value Iteration

In [ ]:
from scipy.special import logsumexp
from scipy.special import softmax

def updatePi(Q):
  P = np.zeros([I,J,NX,NS])#; player, action, z,states,
  P[0] = softmax(Q[0],axis=0)
  P[1] = softmax(Q[1],axis=0)
  P[np.isnan(P)] = 0#;
  return P

def updateQ(thetaS,Q,F,#sigmaP,iterpP,iterf,iterc,
            Q_new,trans=trans_gdp,beta = 0.9):

  gamma = 0.5772
    
  v_mcd = Q[0].copy()
  v_other = Q[1].copy()
  #v_mcd = np.stack([iterpP*Q[0,j][iterf]+(1-iterpP)*Q[0,j][iterc] for j in range(J)],axis=0)
  #v_other = np.stack([iterpP*Q[1,j][iterf]+(1-iterpP)*Q[1,j][iterc] for j in range(J)],axis=0)
  v_mcd = gamma + logsumexp(v_mcd,axis=0) #z_new,NS_new
  v_other = gamma + logsumexp(v_other,axis=0) #z_new,NS_new
  if NX==1:
    v_mcd = v_mcd.reshape([1,NX,NS,1]) #z_old,NS_new
    v_other = v_other.reshape([1,NX,NS,1])    
  else:
    v_mcd = np.dot(trans,v_mcd).reshape([1,NX,NS,1]) #z_old,NS_new
    v_other = np.dot(trans,v_other).reshape([1,NX,NS,1])
  v_mcd = np.kron(v_mcd,np.ones([J,1,1,1])) #a_old,z_old,NS_new,1
  v_other = np.kron(v_other,np.ones([J,1,1,1])) #a_old,z_old,NS_new,1

  v_temp_mcd = np.matmul(F[0],v_mcd)[:,:,:,0] # gdp_old, ns_old, (ns_new \times ns_new), gdp_new
  v_temp_other = np.matmul(F[1],v_other)[:,:,:,0] 

  Q_new[0] = Q_new[0] + beta*(v_temp_mcd) 
  Q_new[1] = Q_new[1] + beta*(v_temp_other)
  return Q_new

theta23 = trans_gdp[:,0]#np.array([0.95, 0.05]+ [0.9])#, 0.01] )#])#
trans = Dynamic(theta23)
#sigma,_,(iterp,iterf,iterc) = SigmaLambda(theta23)
#sigmaP = np.kron(sigma.reshape([NX,obser,1]),np.ones([1,1,NS]))
#iterpP = np.kron(iterp.reshape([1,NX,obser,1]),np.ones([J,1,1,NS]))

import time

def ValueIteration(thetaS,#sigmaP=sigmaP,iterpP=iterpP,iterf=iterf,iterc=iterc,
                   trans=trans_gdp,error = 5e-3,beta = 0.9):
  Pi0 = 1/3*np.ones([I,J,NX,NS])
  F0= updateF(Pi0)
  Q0 = np.zeros([I,J,NX,NS])
  Pi_old = Pi0
  F_old = F0
  Q_old = Q0

  #x_old = np.linspace(0,1,num=NX).reshape([-1,1])
  R_new = np.zeros([I,J,NX,NS])
  if NX==1:
    thetaS_mcd = np.reshape(np.array([thetaS[0],thetaS[1],thetaS[2],thetaS[3],thetaS[4],thetaS[5],thetaS[6]]),[-1,1])#np.reshape(thetaS[0:int(len(thetaS)/2)-2],[-1,1])
    thetaS_other = np.reshape(np.array([thetaS[7],thetaS[8],thetaS[9],thetaS[10],thetaS[11],thetaS[12],thetaS[13]]),[-1,1])#np.reshape(thetaS[int(len(thetaS)/2):len(thetaS)-2],[-1,1])
  else:
    thetaS_mcd = np.reshape(np.array([thetaS[0],thetaS[2],thetaS[3],thetaS[4],thetaS[5],thetaS[6],thetaS[7]]),[-1,1])#np.reshape(thetaS[0:int(len(thetaS)/2)-2],[-1,1])
    thetaS_other = np.reshape(np.array([thetaS[8],thetaS[10],thetaS[11],thetaS[12],thetaS[13],thetaS[14],thetaS[15]]),[-1,1])#np.reshape(thetaS[int(len(thetaS)/2):len(thetaS)-2],[-1,1])
  
  r_mcd_a2 = np.stack([Ni,Ni*Ni,Ni*Nj,Ni*dz1,Ni*dz2,Ni*dz3, -1* np.ones(NS)],axis=1).dot(thetaS_mcd)
  r_mcd_a1 = np.stack([Ni,Ni*Ni,Ni*Nj,Ni*dz1,Ni*dz2,Ni*dz3, 0* np.ones(NS)],axis=1).dot(thetaS_mcd)
  #Ni_temp = (Ni*Ni).reshape([1,-1])
  r_other_a2 = np.stack([Ni,Ni*Ni,Ni*Nj,Ni*dz1,Ni*dz2,Ni*dz3, -1* np.ones(NS)],axis=1).dot(thetaS_other)
  r_other_a1 = np.stack([Ni,Ni*Ni,Ni*Nj,Ni*dz1,Ni*dz2,Ni*dz3, 0* np.ones(NS)],axis=1).dot(thetaS_other)
  
  if NX==1:
    R_new[0,2] =   np.ones([NX,1]).dot(r_mcd_a2.T)
    R_new[0,1] =   np.ones([NX,1]).dot(r_mcd_a1.T)#+\

    R_new[1,2] =  np.ones([NX,1]).dot(r_other_a2.T)#+\
    R_new[1,1] =  np.ones([NX,1]).dot(r_other_a1.T)#+\
  else:
    R_new[0,2,0] =   r_mcd_a2.T[0]
    R_new[0,2,1] = R_new[0,2,0].copy() + thetaS[1]*Ni
    R_new[0,1,0] =   r_mcd_a1.T[0]
    R_new[0,1,1] = R_new[0,1,0].copy() +thetaS[1]*Ni

    R_new[1,2,0] = r_other_a2.T[0]
    R_new[1,2,1] = R_new[1,2,0].copy() + thetaS[9]*Ni
    R_new[1,1,0] =  r_other_a1.T[0]
    R_new[1,1,1] = R_new[1,1,0].copy()+thetaS[9]*Ni

  R_new[0,0] = R_new[0,1].copy()
  R_new[1,0] = R_new[1,1].copy()
  error1 = np.inf
  for iter in range(1000):
    # time_start = time.time()
    Q_new = updateQ(thetaS,Q_old,F_old,#sigmaP,iterpP,iterf,iterc,
                    Q_new = R_new.copy(),trans=trans,beta=beta) #I, J, NS
    Pi_new = updatePi(Q_new)#I,J,NS
    #time_start1 = time.time()
    #print(time_start1-time_start)
    F_new = updateF(Pi_new) # I, J, NS,NS
    #print(time.time()-time_start1)
    if iter>0:
      error1 = np.max(np.abs(Q_new-Q_old))/np.max(np.abs(Q_old))
    # error2 = np.max(np.abs(Pi_new-Pi_old))/np.max(np.abs(Pi_old))
    # error3 = np.max(np.abs(F_new-F_old))/np.max(np.abs(F_old))
    # print('iteration {} --> e1:{:.4f}({:.4f}), e2:{:.4f}({:.4f}), e3:{:.4f}({:.4f}), time:{}'.format(iter,error1,np.max(np.abs(Q_old)),error2,np.max(np.abs(Pi_old)),error3,np.max(np.abs(F_old)),time.time()-time_start))
    # #print('iteration {} --> e1:{},time:{}'.format(iter,error1,time.time()-time_start))

    if error1<error :#and error2<error and error3<error:
      #print('iteration {} --> e1:{},e2:{},e3:{}'.format(iter,error1,error2,error3))
      break
    Q_old = Q_new.copy()
    Pi_old = Pi_new.copy()
    F_old = F_new.copy()
  return Q_new, Pi_new,F_new

thetaS = np.zeros((7+1)*2)
# thetaS = np.array([(3.9488+2.4672)/2,-2.5972,-2.5972,-0.4203,0.0931,0.1513,0.0730,12.5872,
#                    (1.3230+1.1637)/2,-0.5400,-0.5400,-0.1731,0.0596,0.0413,0.0116,7.6820])
Q_new, Pi_new,F_new = ValueIteration(thetaS)

##5. log-likelihood

In [ ]:
import pickle
#with open(file_name+'/transition.txt','rb') as fp:

with open('/content/drive/MyDrive/2022_SUMMER/Research_Burger/Picture/transition.txt','rb') as fp:
  haha = pickle.load(fp)
print(haha)
for i in [2012,2021]:
  for j in ['year','quater','month']:
    print(i,j,haha[i][j][0,0]/np.sum(haha[i][j][0]),haha[i][j][1,0]/np.sum(haha[i][j][1]))
    haha[i][j] = np.array([haha[i][j][0,0]/np.sum(haha[i][j][0]),haha[i][j][1,0]/np.sum(haha[i][j][1])])
haha

{2012: {'year': array([[ 5,  3],
       [ 3, 23]]), 'quater': array([[ 18,   4],
       [  4, 113]]), 'month': array([[ 49,   4],
       [  4, 362]])}, 2021: {'year': array([[ 4,  3],
       [ 3, 24]]), 'quater': array([[ 17,   3],
       [  3, 116]]), 'month': array([[ 47,   3],
       [  3, 366]])}}
2012 year 0.625 0.11538461538461539
2012 quater 0.8181818181818182 0.03418803418803419
2012 month 0.9245283018867925 0.01092896174863388
2021 year 0.5714285714285714 0.1111111111111111
2021 quater 0.85 0.025210084033613446
2021 month 0.94 0.008130081300813009


{2012: {'year': array([0.625     , 0.11538462]),
  'quater': array([0.81818182, 0.03418803]),
  'month': array([0.9245283 , 0.01092896])},
 2021: {'year': array([0.57142857, 0.11111111]),
  'quater': array([0.85      , 0.02521008]),
  'month': array([0.94      , 0.00813008])}}

In [ ]:
tertile_other = np.hstack([tertile,tertile,tertile,tertile])

In [ ]:
disc_gdp_other = np.hstack([disc_gdp for i in range(4)])
disc_gdp_mcd = disc_gdp.copy()
z_data = disc_gdp[tertile==mk].reshape([-1,TimePeriod]).T
x_data = (1/obser)*np.ones(z_data.shape[1])

In [ ]:
theta23,trans_gdp

(array([0.33333333, 0.06451613]), array([[0.33333333, 0.66666667],
        [0.06451613, 0.93548387]]))

In [ ]:
z_data.shape

(35, 134)

In [ ]:
from scipy.optimize import minimize
lower_bound = 1e-3
init_x0 = theta23*0+0.1
def func1(theta23):
  trans = Dynamic(theta23)
  res = -np.sum(np.log(trans[z_data[0:TimePeriod-1],z_data[1:TimePeriod]]))
  print(theta23,res)
  return res
res1 = minimize(func1,init_x0,bounds=[(lower_bound,1-lower_bound),(lower_bound,1-lower_bound)],options={'maxiter':1000},tol=1e-6)
trans = Dynamic(res1.x)
print(trans)

[0.1 0.1] 1363.3067894262167
[0.10000001 0.1       ] 1363.306779003995
[0.1        0.10000001] 1363.3068058039962
[0.999 0.001] 3706.578840875431
[0.99899999 0.001     ] 3706.576162230159
[0.999      0.00100001] 3706.5761997877307
[0.46843163 0.05942744] 1265.6217511642965
[0.46843164 0.05942744] 1265.6217533453714
[0.46843163 0.05942745] 1265.621747382548
[0.40718372 0.06860854] 1254.789630513466
[0.40718373 0.06860854] 1254.789631743362
[0.40718372 0.06860855] 1254.7896331738
[0.32284186 0.06405715] 1249.6944267541658
[0.32284187 0.06405715] 1249.6944265612435
[0.32284186 0.06405716] 1249.6944264361587
[0.33442264 0.06447398] 1249.587636640006
[0.33442265 0.06447398] 1249.5876366596794
[0.33442264 0.06447399] 1249.5876366109794
[0.33322446 0.0646089 ] 1249.5868100303549
[0.33322447 0.0646089 ] 1249.5868100283853
[0.33322446 0.06460891] 1249.586810094127
[[0.33322446 0.66677554]
 [0.0646089  0.9353911 ]]


In [ ]:
# from scipy.optimize import minimize
# lower_bound = 1e-3
# a1 = 0.8993
# a2 = 0.008
# def func1(theta23):
#   theta23 = np.array([a1,a2,#np.array([haha[2021]['month'][0],haha[2021]['month'][1],
#                       theta23[0]])#,theta23[1]])
#   sigma_val,_,_ = SigmaLambda(theta23,z_old = z_data,x_old=x_data,T=TimePeriod)
#   #sigma_val[np.isnan(sigma_val)] = 0
#   res = -np.sum(np.log(sigma_val[np.where(sigma_val!=0)]))

#   print(theta23,res)
#   return res /(5*MT)

# from scipy import optimize
# rranges = [(0.5+lower_bound,1-lower_bound)]#,(lower_bound,0.5-lower_bound)]# *2
# init_x0 = [0.9]#,0.1]#np.array([trans_gdp[0][0],trans_gdp[1,0],0.9+lower_bound])# ,0.5-lower_bound])#np.array([2.47142617e-01, 6.98155974e-02, 9.50769569e-01, 1.00000000e-06])#
# print('start from:',init_x0)
# res1 = minimize(func1,init_x0,bounds = rranges,options={'maxiter':1000},tol=1e-5)
# print(res1,res1.fun)#*5*MT)
# print(Dynamic(np.array([a1,a2,#np.array([haha[2021]['month'][0],haha[2021]['month'][1],
#                       res1.x[0]])))#,res1.x[1]])))
# # resbrute = optimize.brute(func1, rranges, finish=None,Ns=10)
# # print(resbrute[0])

In [ ]:
norm_thetaS = 10
# sigma1,_,(iterp1,iterf1,iterc1) = SigmaLambda(np.array([a1,a2,#np.array([haha[2021]['month'][0],haha[2021]['month'][1],
#                       res1.x[0]]))
# sigmaP1 = np.kron(sigma.reshape([NX,obser,1]),np.ones([1,1,NS]))
# iterpP1 = np.kron(iterp.reshape([1,NX,obser,1]),np.ones([J,1,1,NS]))

# _,_,(iterp2,iterf2,iterc2) = SigmaLambda(np.array([a1,a2,#np.array([haha[2021]['month'][0],haha[2021]['month'][1],
#                       res1.x[0]]),z_old = z_data,x_old=x_data,T=TimePeriod)
# #x_blief = (x_blief.T).reshape(-1)
# iterp2 = (iterp2.T).reshape(-1)
# iterf2 = (iterf2.T).reshape(-1)
# iterc2 = (iterc2.T).reshape(-1)
# iterp2_other = np.hstack([iterp2 for i in range(4)])
# iterf2_other = np.hstack([iterf2 for i in range(4)])
# iterc2_other = np.hstack([iterc2 for i in range(4)])

def func(thetaS,trans=trans):
  #thetaS = np.hstack([thetaS[0:7],0/norm_thetaS,thetaS[7:15],0/norm_thetaS,thetaS[15]])
  _, Pi_new,_ = ValueIteration(thetaS*norm_thetaS,#sigmaP=sigmaP1,iterpP=iterpP1,iterf=iterf1,iterc=iterc1
                              trans=trans)
  #print(np.unique(a_mcd))
  if NX==1:
    res = -np.sum(np.log(  Pi_new[0,ai_mcd[tertile==mk],0,stateID_mcd[tertile==mk]]))-\
           np.sum(np.log(  Pi_new[1,ai_other[tertile_other==mk],0,stateID_other[tertile_other==mk]]))
  else:
    res = -np.sum(np.log(  Pi_new[0,ai_mcd[tertile==mk],0,stateID_mcd[tertile==mk]])+\
                np.log(  Pi_new[0,ai_mcd[tertile==mk],1,stateID_mcd[tertile==mk]]))
    res = res -np.sum(np.log(  Pi_new[1,ai_other[tertile_other==mk],0,stateID_other[tertile_other==mk]])+\
                    np.log(  Pi_new[1,ai_other[tertile_other==mk],1,stateID_other[tertile_other==mk]]))
  print(thetaS*norm_thetaS,res)
  return res/5/MT
from scipy.optimize import minimize

# thetaS = np.array([(1.916+2.1589)/2,-0.5025,-0.0981,0.0511,0.0562,0.0516,12.6968,
#                    (0.9989+1.4974)/2,-0.4957,-0.1537,0.0288,0.0586,0.0762,7.5245])/norm_thetaS
thetaS = np.array([2.21288709, -0.50203196, -0.09316142,  0.05154506,  0.05558539,
        0.0533467 , 13.4592347 ,  1.34683594, -0.49251677, -0.14927171,
        0.02879205,  0.0577639 ,  0.07764627,  7.52410491])/norm_thetaS
# thetaS = np.zeros((7)*2)
print('start from:',thetaS*norm_thetaS )
res22 = minimize(func,thetaS,options={'maxiter':4000},tol=1e-4)
print(res22,res22.fun*5*MT) # 28min

start from: [ 2.21288709 -0.50203196 -0.09316142  0.05154506  0.05558539  0.0533467
 13.4592347   1.34683594 -0.49251677 -0.14927171  0.02879205  0.0577639
  0.07764627  7.52410491]
[ 2.21288709 -0.50203196 -0.09316142  0.05154506  0.05558539  0.0533467
 13.4592347   1.34683594 -0.49251677 -0.14927171  0.02879205  0.0577639
  0.07764627  7.52410491] 15460.821637592486
[ 2.21288724 -0.50203196 -0.09316142  0.05154506  0.05558539  0.0533467
 13.4592347   1.34683594 -0.49251677 -0.14927171  0.02879205  0.0577639
  0.07764627  7.52410491] 15460.821637592588
[ 2.21288709 -0.50203181 -0.09316142  0.05154506  0.05558539  0.0533467
 13.4592347   1.34683594 -0.49251677 -0.14927171  0.02879205  0.0577639
  0.07764627  7.52410491] 15460.821637592871
[ 2.21288709 -0.50203196 -0.09316127  0.05154506  0.05558539  0.0533467
 13.4592347   1.34683594 -0.49251677 -0.14927171  0.02879205  0.0577639
  0.07764627  7.52410491] 15460.821637592455
[ 2.21288709 -0.50203196 -0.09316142  0.05154521  0.05558539  

In [ ]:
import numpy as np

haha = res22.x*norm_thetaS
if NX==1:
  print('{:.4f}&{:.4f}'.format(haha[0],haha[7]))
  print('{:.4f}&{:.4f}'.format(haha[1],haha[8]))
  print('{:.4f}&{:.4f}'.format(haha[2],haha[9]))
  print('{:.4f}&{:.4f}'.format(haha[3],haha[10]))
  print('{:.4f}&{:.4f}'.format(haha[4],haha[11]))
  print('{:.4f}&{:.4f}'.format(haha[5],haha[12]))
  print('{:.4f}&{:.4f}'.format(haha[6],haha[13]))
else:

  print('{:.4f}&{:.4f}'.format(haha[0],haha[8]))
  print('{:.4f}&{:.4f}'.format(haha[1],haha[9]))
  print('{:.4f}&{:.4f}'.format(haha[2],haha[10]))
  print('{:.4f}&{:.4f}'.format(haha[3],haha[11]))
  print('{:.4f}&{:.4f}'.format(haha[4],haha[12]))
  print('{:.4f}&{:.4f}'.format(haha[5],haha[13]))
  print('{:.4f}&{:.4f}'.format(haha[6],haha[14]))
  print('{:.4f}&{:.4f}'.format(haha[7],haha[15]))



2.2129&1.3468
-0.5020&-0.4925
-0.0932&-0.1493
0.0515&0.0288
0.0556&0.0578
0.0533&0.0776
13.4592&7.5241
